# UAS

In [1]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.svm import SVC

import pandas as pd
import warnings
import joblib
import nltk
import re

nltk.download('stopwords')
nltk.download('punkt')
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/MeAdila/ppw/gh-pages/berita-BBC.csv')
df

,Judul Berita,Isi Berita,Kategori Berita
0,Viral penemuan lima mayat di Kampus Unpri Meda...,Viral penemuan lima mayat di Kampus Unpri Meda...,Indonesia
1,Debat capres berpengaruh terhadap pemilih meng...,Debat capres berpengaruh terhadap pemilih meng...,Indonesia
2,"Pilpres 2024: Apa janji capres soal hukum, HAM...","Pilpres 2024: Apa janji capres soal hukum, HAM...",Indonesia
3,TikTok Shop buka lagi setelah akuisisi Tokoped...,TikTok Shop buka lagi setelah akuisisi Tokoped...,Indonesia
4,Siapa yang menyebarkan narasi kebencian dan ho...,Siapa yang menyebarkan narasi kebencian dan ho...,Indonesia
...,...,...,...
715,Demonstrasi anti-LGBT warnai konser Coldplay d...,Demonstrasi anti-LGBT warnai konser Coldplay d...,Viral
716,"SEA Games 2023 'dihujani kritik', apa yang seb...","SEA Games 2023 'dihujani kritik', apa yang seb...",Viral
717,"Fenomena ‘garasi’ di jalan umum, mengapa selal...","SEA Games 2023 'dihujani kritik', apa yang seb...",Viral
718,Presiden Jokowi ambil alih perbaikan jalan di ...,"SEA Games 2023 'dihujani kritik', apa yang seb...",Viral


In [3]:
df.isnull().sum()

Judul Berita       0
Isi Berita         0
Kategori Berita    0
dtype: int64

In [5]:
def cleaning(text):
  text = re.sub(r'[^a-zA-Z\s]', '', text).strip()
  return text

df['Cleaning'] = df['Isi Berita'].apply(cleaning)
df['Cleaning']

0      Viral penemuan lima mayat di Kampus Unpri Meda...
1      Debat capres berpengaruh terhadap pemilih meng...
2      Pilpres  Apa janji capres soal hukum HAM korup...
3      TikTok Shop buka lagi setelah akuisisi Tokoped...
4      Siapa yang menyebarkan narasi kebencian dan ho...
                             ...                        
715    Demonstrasi antiLGBT warnai konser Coldplay di...
716    SEA Games  dihujani kritik apa yang sebenarnya...
717    SEA Games  dihujani kritik apa yang sebenarnya...
718    SEA Games  dihujani kritik apa yang sebenarnya...
719    Polda Lampung hentikan kasus TikToker Bima pak...
Name: Cleaning, Length: 720, dtype: object

In [7]:
def tokenizer(text):
  text = text.lower()
  return word_tokenize(text)

df['Tokenizing'] = df['Cleaning'].apply(tokenizer)
df['Tokenizing']

0      [viral, penemuan, lima, mayat, di, kampus, unp...
1      [debat, capres, berpengaruh, terhadap, pemilih...
2      [pilpres, apa, janji, capres, soal, hukum, ham...
3      [tiktok, shop, buka, lagi, setelah, akuisisi, ...
4      [siapa, yang, menyebarkan, narasi, kebencian, ...
                             ...                        
715    [demonstrasi, antilgbt, warnai, konser, coldpl...
716    [sea, games, dihujani, kritik, apa, yang, sebe...
717    [sea, games, dihujani, kritik, apa, yang, sebe...
718    [sea, games, dihujani, kritik, apa, yang, sebe...
719    [polda, lampung, hentikan, kasus, tiktoker, bi...
Name: Tokenizing, Length: 720, dtype: object

In [8]:
corpus = stopwords.words('indonesian')

def stopwordText(words):
 return [word for word in words if word not in corpus]

df['Stopword Removal'] = df['Tokenizing'].apply(stopwordText)

# Gabungkan kembali token menjadi kalimat utuh
df['Full Text'] = df['Stopword Removal'].apply(lambda x: ' '.join(x))
df['Full Text']

0      viral penemuan mayat kampus unpri medan korban...
1      debat capres berpengaruh pemilih mengambangsum...
2      pilpres janji capres hukum ham korupsi pelayan...
3      tiktok shop buka akuisisi tokopedia pedagang a...
4      menyebarkan narasi kebencian hoaks rohingya me...
                             ...                        
715    demonstrasi antilgbt warnai konser coldplay ja...
716    sea games dihujani kritik kambojasumber gambar...
717    sea games dihujani kritik kambojasumber gambar...
718    sea games dihujani kritik kambojasumber gambar...
719    polda lampung hentikan tiktoker bima pakar uns...
Name: Full Text, Length: 720, dtype: object

In [10]:
def tfidf(dokumen, category):
  vectorizer = TfidfVectorizer()
  x = vectorizer.fit_transform(dokumen).toarray()
  terms = vectorizer.get_feature_names_out()

  final_tfidf = pd.DataFrame(x, columns=terms)
  final_tfidf.insert(0, 'Dokumen', dokumen)
  final_tfidf.insert(len(final_tfidf.columns),'Kategori Berita', category)

  return (vectorizer, final_tfidf)

tfidf_vectorizer, final_tfidf = tfidf(df['Full Text'], df['Kategori Berita'])
final_tfidf

,Dokumen,aaron,abaaba,abad,abadi,abang,abasan,abbas,abdirahim,abdul,...,zpemilu,zprabowo,zsumber,zubairnamun,zuckerberg,zulfacici,zulhas,zuliansyah,zulkifli,Kategori Berita
0,viral penemuan mayat kampus unpri medan korban...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,Indonesia
1,debat capres berpengaruh pemilih mengambangsum...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.014528,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,Indonesia
2,pilpres janji capres hukum ham korupsi pelayan...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.045115,0.000000,Indonesia
3,tiktok shop buka akuisisi tokopedia pedagang a...,0.0,0.0,0.0,0.0,0.089547,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.029849,0.000000,0.023882,Indonesia
4,menyebarkan narasi kebencian hoaks rohingya me...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,Indonesia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,demonstrasi antilgbt warnai konser coldplay ja...,0.0,0.0,0.0,0.0,0.000000,0.0,0.019064,0.0,0.033371,...,0.000000,0.0,0.0,0.0,0.0,0.019064,0.000000,0.000000,0.000000,Viral
716,sea games dihujani kritik kambojasumber gambar...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,Viral
717,sea games dihujani kritik kambojasumber gambar...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,Viral
718,sea games dihujani kritik kambojasumber gambar...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,Viral


In [11]:
X = final_tfidf.drop(['Dokumen', 'Kategori Berita'], axis=1)
X

,aaron,abaaba,abad,abadi,abang,abasan,abbas,abdirahim,abdul,abdullah,...,zona,zpemilu,zprabowo,zsumber,zubairnamun,zuckerberg,zulfacici,zulhas,zuliansyah,zulkifli
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.014528,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.045115,0.000000
3,0.0,0.0,0.0,0.0,0.089547,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.029849,0.000000,0.023882
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,0.0,0.0,0.0,0.0,0.000000,0.0,0.019064,0.0,0.033371,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.019064,0.000000,0.000000,0.000000
716,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
717,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
718,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000


In [12]:
y = df['Kategori Berita']
y

0       Indonesia
1       Indonesia
2       Indonesia
3       Indonesia
4       Indonesia
          ...    
715         Viral
716         Viral
717         Viral
718         Viral
719         Viral
Name: Kategori Berita, Length: 720, dtype: object

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [14]:
def find_best_lda(Xtrain, Xtest, n_components, alpha, beta):
  looping = 1
  best = {'k' : 0, 'alpha' : 0, 'beta' : 0, 'accuracy' : 0, 'model': '', 'lda' : '', 'lda_Xtrain' : '', 'lda_Xtest' : ''}
  history = pd.DataFrame(columns=["Pengujian Ke", "K", "Alpha", "Beta", "Accuracy"])

  # Menambahkan tqdm pada loop terluar
  for k in n_components:
    for a in alpha:
      for b in beta:
        lda = LatentDirichletAllocation(n_components=k, doc_topic_prior=a, topic_word_prior=b)
        lda_Xtrain = lda.fit_transform(Xtrain)
        lda_Xtest = lda.transform(Xtest)

        # Membuat model Naive Bayes
        model = GaussianNB()

        # Melatih model pada data pelatihan
        model.fit(lda_Xtrain, y_train)

        # Melakukan prediksi pada data pengujian
        y_pred = model.predict(lda_Xtest)

        # Menghitung akurasi
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Jumlah Topik: {k}, Alpha: {a}, Beta: {b}, Accuracy: {accuracy}")

        if accuracy > best['accuracy']:
          best['accuracy'] = accuracy
          best['k'] = k
          best['alpha'] = a
          best['beta'] = b
          best['model'] = model
          best['lda'] = lda
          best['lda_Xtrain'] = lda_Xtrain
          best['lda_Xtest'] = lda_Xtest

        history.loc[len(history)] = [f"Pengujian Ke- {looping}", k, a, b, accuracy]
        looping += 1

  return (best, history)

k = [3, 4, 5]
alpha = [0.3, 0.4]
beta = [0.1, 0.2]
best_param, history = find_best_lda(X_train, X_test, k, alpha, beta)

Jumlah Topik: 3, Alpha: 0.3, Beta: 0.1, Accuracy: 0.4537037037037037
Jumlah Topik: 3, Alpha: 0.3, Beta: 0.2, Accuracy: 0.4212962962962963
Jumlah Topik: 3, Alpha: 0.4, Beta: 0.1, Accuracy: 0.38425925925925924
Jumlah Topik: 3, Alpha: 0.4, Beta: 0.2, Accuracy: 0.47685185185185186
Jumlah Topik: 4, Alpha: 0.3, Beta: 0.1, Accuracy: 0.4583333333333333
Jumlah Topik: 4, Alpha: 0.3, Beta: 0.2, Accuracy: 0.6064814814814815
Jumlah Topik: 4, Alpha: 0.4, Beta: 0.1, Accuracy: 0.4537037037037037
Jumlah Topik: 4, Alpha: 0.4, Beta: 0.2, Accuracy: 0.47685185185185186
Jumlah Topik: 5, Alpha: 0.3, Beta: 0.1, Accuracy: 0.4398148148148148
Jumlah Topik: 5, Alpha: 0.3, Beta: 0.2, Accuracy: 0.4444444444444444
Jumlah Topik: 5, Alpha: 0.4, Beta: 0.1, Accuracy: 0.5046296296296297
Jumlah Topik: 5, Alpha: 0.4, Beta: 0.2, Accuracy: 0.47685185185185186


In [15]:
best_param

{'k': 4,
 'alpha': 0.3,
 'beta': 0.2,
 'accuracy': 0.6064814814814815,
 'model': GaussianNB(),
 'lda': LatentDirichletAllocation(doc_topic_prior=0.3, n_components=4,
                           topic_word_prior=0.2),
 'lda_Xtrain': array([[0.02398925, 0.0226888 , 0.92768454, 0.02563741],
        [0.0246474 , 0.02462528, 0.9261775 , 0.02454982],
        [0.92604649, 0.02396538, 0.02470211, 0.02528602],
        ...,
        [0.93406897, 0.02443753, 0.02049246, 0.02100105],
        [0.02680311, 0.02614568, 0.02620743, 0.92084378],
        [0.1455434 , 0.02298646, 0.80366175, 0.02780839]]),
 'lda_Xtest': array([[0.93332665, 0.02169687, 0.02277978, 0.0221967 ],
        [0.1958272 , 0.0235907 , 0.02545582, 0.75512628],
        [0.1455434 , 0.02298646, 0.80366175, 0.02780839],
        [0.1455434 , 0.02298646, 0.80366175, 0.02780839],
        [0.9462094 , 0.01749721, 0.01810728, 0.01818612],
        [0.02374227, 0.02230438, 0.02566893, 0.92828443],
        [0.9223189 , 0.02456714, 0.02666677, 0

In [16]:
history

,Pengujian Ke,K,Alpha,Beta,Accuracy
0,Pengujian Ke- 1,3,0.3,0.1,0.453704
1,Pengujian Ke- 2,3,0.3,0.2,0.421296
2,Pengujian Ke- 3,3,0.4,0.1,0.384259
3,Pengujian Ke- 4,3,0.4,0.2,0.476852
4,Pengujian Ke- 5,4,0.3,0.1,0.458333
5,Pengujian Ke- 6,4,0.3,0.2,0.606481
6,Pengujian Ke- 7,4,0.4,0.1,0.453704
7,Pengujian Ke- 8,4,0.4,0.2,0.476852
8,Pengujian Ke- 9,5,0.3,0.1,0.439815
9,Pengujian Ke- 10,5,0.3,0.2,0.444444


In [17]:
history.to_csv("history.csv", index=False)

In [18]:
lda = best_param['lda']
lda_x_train = best_param['lda_Xtrain']
lda_x_test = best_param['lda_Xtest']

In [20]:
topik_columns = [f"Topik {i}" for i in range(1, best_param['k']+1)]
dokumen = final_tfidf['Dokumen']
output_proporsi_TD = pd.DataFrame(lda_x_train, columns=topik_columns)
output_proporsi_TD.insert(0,'Dokumen', dokumen)
output_proporsi_TD.insert(len(output_proporsi_TD.columns),'Kategori Berita', final_tfidf['Kategori Berita'])
output_proporsi_TD

,Dokumen,Topik 1,Topik 2,Topik 3,Topik 4,Kategori Berita
0,viral penemuan mayat kampus unpri medan korban...,0.023989,0.022689,0.927685,0.025637,Indonesia
1,debat capres berpengaruh pemilih mengambangsum...,0.024647,0.024625,0.926178,0.024550,Indonesia
2,pilpres janji capres hukum ham korupsi pelayan...,0.926046,0.023965,0.024702,0.025286,Indonesia
3,tiktok shop buka akuisisi tokopedia pedagang a...,0.025135,0.024860,0.025170,0.924835,Indonesia
4,menyebarkan narasi kebencian hoaks rohingya me...,0.026803,0.026146,0.026207,0.920844,Indonesia
...,...,...,...,...,...,...
499,demonstrasi antilgbt warnai konser coldplay ja...,0.026206,0.025492,0.922501,0.025801,Viral
500,sea games dihujani kritik kambojasumber gambar...,0.021713,0.933629,0.021149,0.023509,Viral
501,sea games dihujani kritik kambojasumber gambar...,0.934069,0.024438,0.020492,0.021001,Viral
502,sea games dihujani kritik kambojasumber gambar...,0.026803,0.026146,0.026207,0.920844,Viral


In [21]:
output_proporsi_TD.to_csv('reduksi dimensi.csv', index=False)

In [22]:
# Output distribusi kata pada topik
distribusi_kata_topik = pd.DataFrame(lda.components_)
distribusi_kata_topik

,0,1,2,3,4,5,6,7,8,9,...,12403,12404,12405,12406,12407,12408,12409,12410,12411,12412
0,0.638957,0.200043,0.293514,0.202084,1.005846,0.360933,0.295034,0.404114,0.364381,0.419379,...,0.814166,0.200064,0.200133,0.200133,0.200054,0.349130,0.295034,0.468518,0.200035,0.923773
1,0.200045,0.200051,0.200109,0.200071,0.200028,0.200047,0.200094,0.200046,0.200111,0.200063,...,0.200063,0.200071,0.200044,0.200044,0.200057,0.200083,0.200094,0.200045,0.200027,0.200026
2,0.200034,0.200044,0.200052,0.338904,0.200025,0.200043,0.200100,0.200041,0.528857,0.200047,...,0.200061,0.200074,0.200360,0.200360,0.337909,1.656821,0.200100,0.200040,0.200028,0.200025
3,0.200034,0.329872,0.200055,0.200106,0.200024,0.200061,0.200092,0.200039,0.200116,0.200047,...,0.532346,0.301486,0.290796,0.290796,0.200045,0.200080,0.200092,0.200037,0.605941,0.200023


In [23]:
# Membuat model Naive Bayes
model = best_param['model']

# Melakukan prediksi pada data pengujian
y_pred = model.predict(lda_x_test)

# Menghitung akurasi
accuracy = accuracy_score(y_test, y_pred)
print("Akurasi:", accuracy)

# Menampilkan laporan klasifikasi
print("Laporan Klasifikasi:")
print(classification_report(y_test, y_pred))

# Menampilkan matriks kebingungan
confusion = confusion_matrix(y_test, y_pred)
print("Confusion Matriks:")
print(confusion)

Akurasi: 0.6064814814814815
Laporan Klasifikasi:
              precision    recall  f1-score   support

       Dunia       0.65      0.59      0.62        73
   Indonesia       0.71      0.53      0.60        80
       Viral       0.51      0.73      0.60        63

    accuracy                           0.61       216
   macro avg       0.62      0.61      0.61       216
weighted avg       0.63      0.61      0.61       216

Confusion Matriks:
[[43 11 19]
 [12 42 26]
 [11  6 46]]


In [24]:
data = ["Penelitian ini menggabungkan konsep kecerdasan buatan dengan algoritma penjadwalan dalam upaya meningkatkan efisiensi produksi dalam lingkungan manufaktur. Kami memperkenalkan pendekatan yang memanfaatkan kecerdasan komputasional, yaitu algoritma optimasi berbasis swarm intelligence, seperti algoritma PSO (Particle Swarm Optimization) dan algoritma ACO (Ant Colony Optimization). Tujuan utama penelitian ini adalah untuk mengoptimalkan jadwal produksi dengan meminimalkan waktu produksi dan biaya, sambil mempertimbangkan berbagai kendala produksi seperti kapasitas mesin, waktu pemrosesan, dan persyaratan bahan baku. Melalui eksperimen dan simulasi, kami membandingkan hasil dari algoritma swarm intelligence dengan pendekatan konvensional. Hasilnya menunjukkan bahwa algoritma PSO dan ACO dapat menghasilkan jadwal produksi yang lebih efisien, dengan waktu produksi yang lebih pendek dan biaya yang lebih rendah. Selain itu, algoritma ini mampu beradaptasi dengan perubahan dalam lingkungan produksi dan menghasilkan jadwal yang optimal bahkan dalam situasi yang kompleks. Penelitian ini menunjukkan potensi besar dari penggunaan kecerdasan komputasional dalam meningkatkan efisiensi dan produktivitas dalam industri manufaktur. Hasil penelitian ini dapat digunakan sebagai dasar untuk mengembangkan sistem penjadwalan cerdas yang dapat diterapkan dalam berbagai industri."]
a = tfidf_vectorizer.transform(data).toarray()
b = lda.transform(a)
model.predict(b)

array([' Indonesia'], dtype='<U10')

In [25]:
joblib.dump(lda, "lda.pkl")
joblib.dump(model, "naive bayes.pkl")

['naive bayes.pkl']

In [26]:
# Membuat model Naive Bayes
nb = GaussianNB()

# Melatih model nbada data pelatihan
nb.fit(X_train, y_train)

# Melakukan prediksi pada data pengujian
y_pred_nb = nb.predict(X_test)

# Menghitung akurasi
accuracy = accuracy_score(y_test, y_pred_nb)
print("Akurasi:", accuracy)

# Menampilkan laporan klasifikasi
print("Laporan Klasifikasi:")
print(classification_report(y_test, y_pred_nb))

# Menampilkan matriks kebingungan
confusion = confusion_matrix(y_test, y_pred_nb)
print("Confusion Matriks:")
print(confusion)

Akurasi: 0.8842592592592593
Laporan Klasifikasi:
              precision    recall  f1-score   support

       Dunia       0.91      0.73      0.81        73
   Indonesia       0.79      0.94      0.86        80
       Viral       1.00      1.00      1.00        63

    accuracy                           0.88       216
   macro avg       0.90      0.89      0.89       216
weighted avg       0.89      0.88      0.88       216

Confusion Matriks:
[[53 20  0]
 [ 5 75  0]
 [ 0  0 63]]


In [27]:
data = ["Penelitian ini menggabungkan konsep kecerdasan buatan dengan algoritma penjadwalan dalam upaya meningkatkan efisiensi produksi dalam lingkungan manufaktur. Kami memperkenalkan pendekatan yang memanfaatkan kecerdasan komputasional, yaitu algoritma optimasi berbasis swarm intelligence, seperti algoritma PSO (Particle Swarm Optimization) dan algoritma ACO (Ant Colony Optimization). Tujuan utama penelitian ini adalah untuk mengoptimalkan jadwal produksi dengan meminimalkan waktu produksi dan biaya, sambil mempertimbangkan berbagai kendala produksi seperti kapasitas mesin, waktu pemrosesan, dan persyaratan bahan baku. Melalui eksperimen dan simulasi, kami membandingkan hasil dari algoritma swarm intelligence dengan pendekatan konvensional. Hasilnya menunjukkan bahwa algoritma PSO dan ACO dapat menghasilkan jadwal produksi yang lebih efisien, dengan waktu produksi yang lebih pendek dan biaya yang lebih rendah. Selain itu, algoritma ini mampu beradaptasi dengan perubahan dalam lingkungan produksi dan menghasilkan jadwal yang optimal bahkan dalam situasi yang kompleks. Penelitian ini menunjukkan potensi besar dari penggunaan kecerdasan komputasional dalam meningkatkan efisiensi dan produktivitas dalam industri manufaktur. Hasil penelitian ini dapat digunakan sebagai dasar untuk mengembangkan sistem penjadwalan cerdas yang dapat diterapkan dalam berbagai industri."]
tfidf_matrix = tfidf_vectorizer.transform(data).toarray()
nb.predict(tfidf_matrix)

array([' Indonesia'], dtype='<U10')

In [28]:
joblib.dump(nb, "Naive Bayes (Asli).pkl")

['Naive Bayes (Asli).pkl']

In [29]:
joblib.dump(tfidf_vectorizer, 'vectorizer.pkl')

['vectorizer.pkl']